In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('ms_marco', 'v2.1')

In [3]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 101093
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 808731
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 101092
    })
})

In [4]:
dataset_train = load_dataset('ms_marco', 'v2.1', split='train')

In [5]:
dataset_train

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 808731
})

In [6]:
def preprocess(example):
    positive_passages=[]
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==1:
            positive_passages.append(example["passages"]["passage_text"][p[0]])
    negative_passages = []
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==0:
            negative_passages.append(example["passages"]["passage_text"][p[0]])
    if (len(positive_passages)>0 and len(negative_passages)>=5):
        positive = positive_passages[0]
        negatives = negative_passages[:5]
        return {
            "query": example["query"],
            "positive": positive,
            "negatives": negatives
        }
    else:
        return {"query": None, "positive": None, "negatives": None}

In [7]:
processed_data_train = dataset_train.map(preprocess, remove_columns=dataset_train.column_names)

In [8]:
processed_data_train = processed_data_train.filter(lambda x: x['query'] is not None and x['positive'] is not None)

In [9]:
contrastive_pairs_train = []
for item in processed_data_train:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_train.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [10]:
len(contrastive_pairs_train)

502824

In [11]:
from torch.utils.data import DataLoader

In [12]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [13]:
contrastive_dataset_train = ContrastiveDataset(contrastive_pairs_train)

In [14]:
data_loader_train = DataLoader(contrastive_dataset_train, batch_size=32, shuffle=True)

In [15]:
len(data_loader_train)

15714

In [16]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [17]:
model = AutoModel.from_pretrained("bert-base-uncased")

In [18]:
from peft import LoraConfig, get_peft_model

In [19]:
lora_config = LoraConfig(
    task_type= "FEATURE_EXTRACTION"
)

In [20]:
lora_model = get_peft_model(model, lora_config)

In [21]:
import torch

In [22]:
def lorentzian_distance(x, y):
    
    dot_product = torch.sum(x * y, dim=-1)
    norm_x = torch.norm(x, dim=-1)
    norm_y = torch.norm(y, dim=-1)
    
    distance = torch.acosh(-dot_product + torch.sqrt((1 + norm_x**2) * (1 + norm_y**2)))
    return distance

In [23]:
def info_nce_loss(anchor_embedding, positive_embedding, negative_embedding, distance_fn):

    pos_dist = distance_fn(anchor_embedding, positive_embedding)
    neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
    
    logits = torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)
    labels = torch.zeros(logits.size(0), dtype=torch.long, device=logits.device)

    loss = torch.nn.CrossEntropyLoss()(logits, labels)
    return loss

In [24]:
def exterior_angle(x_space, y_space, c):
    norm_x_space = torch.norm(x_space, p=2, dim=-1)
    norm_y_space = torch.norm(y_space, p=2, dim=-1)
    x_time = torch.sqrt(1/c + norm_x_space**2)
    y_time = torch.sqrt(1/c + norm_y_space**2)
    dot_product = torch.sum(x_space * y_space, dim=-1)
    lorentz_inner_product =  dot_product - x_time * y_time
    numerator = y_time + x_time * c * lorentz_inner_product
    denominator = norm_x_space * torch.sqrt((c * lorentz_inner_product)**2 - 1)
    ext_angle = torch.acos(numerator / denominator)
    return ext_angle

In [25]:
def entailment_loss(x, y, c=1, K=0.1):
    c = torch.tensor(c)
    K = torch.tensor(K)
    xspace = x
    yspace = y
    aperture = torch.asin(2 * K / (torch.sqrt(c) * torch.norm(xspace, p=2, dim=-1)))
    
    ext_angle = exterior_angle(xspace,yspace,c=c)
    
    loss = torch.max(torch.zeros_like(ext_angle), ext_angle - aperture)
    return loss.mean()

In [26]:
def expm_o(v, c=1.0):
    c = torch.tensor(c)
    vspace = v
    vnorm = torch.norm(v, p=2, dim=-1, keepdim=True)
    xspace = torch.sinh(torch.sqrt(c) * vnorm) * vspace / (torch.sqrt(c) * vnorm)
    batch_min = xspace.min(dim=1, keepdim=True).values
    batch_max = xspace.max(dim=1, keepdim=True).values
    xspace_scaled=(xspace - batch_min) / (batch_max - batch_min)
    return xspace_scaled

In [27]:
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=5e-5)

In [28]:
num_epochs=3

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
device

device(type='cuda')

In [31]:
lora_model = lora_model.to(device)

In [32]:
dataset_val = load_dataset('ms_marco', 'v2.1', split='validation')

In [33]:
processed_data_val = dataset_val.map(preprocess, remove_columns=dataset_val.column_names)

In [34]:
processed_data_val = processed_data_val.filter(lambda x: x['query'] is not None and x['positive'] is not None)

In [35]:
contrastive_pairs_val = []
for item in processed_data_val:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_val.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [36]:
contrastive_dataset_val = ContrastiveDataset(contrastive_pairs_val)

In [37]:
data_loader_val = DataLoader(contrastive_dataset_val, batch_size=32, shuffle=True)

In [38]:
def evaluate_mrr(model1, data_loader_val, distance_fn):
    model1.eval()
    
    total_rr = 0.0
    num_queries = 0

    with torch.no_grad():
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

            anchor_embedding = expm_o(model1(**anchor_input).last_hidden_state[:, 0, :])
            positive_embedding = expm_o(model1(**positive_input).last_hidden_state[:, 0, :])
            negative_embedding = [expm_o(model1(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :]) for neg in negative_texts]

            pos_dist = distance_fn(anchor_embedding, positive_embedding)
            neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
            all_similarities=torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)

            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()  # Reciprocal rank
            num_queries += len(positive_rank)
            
    mrr = total_rr / num_queries
    return mrr

In [39]:
import os
save_dir ="/dss/dsshome1/07/ra65bex2/srawat/v2/0.9hyperbolic"
import time
epoch_metrics = []

In [40]:
for epoch in range(num_epochs):
    start_time = time.time()
    lora_model.train()

    total_loss = 0.0
    entailment_loss_total=0.0
    contrastive_loss_total=0.0
    for batch in data_loader_train:

        anchor_texts = batch[0]
        positive_texts = batch[1]
        negative_texts = batch[2]

        anchor_inputs = tokenizer(anchor_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        positive_inputs = tokenizer(positive_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
        anchor_embedding = expm_o(lora_model(**anchor_inputs).last_hidden_state[:, 0, :])
        positive_embedding = expm_o(lora_model(**positive_inputs).last_hidden_state[:, 0, :])
        negative_embedding = [expm_o(lora_model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :]) for neg in negative_texts]

        contrastive_loss_value = info_nce_loss(anchor_embedding, positive_embedding, negative_embedding, distance_fn=lorentzian_distance)
        
        entailment_loss_value = entailment_loss(anchor_embedding, positive_embedding)
        
        loss = contrastive_loss_value + 0.9*entailment_loss_value
  
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        entailment_loss_total+=entailment_loss_value.item()
        contrastive_loss_total+=contrastive_loss_value.item()
    save_path1 = os.path.join(save_dir, f"hyperbolic_lora_checkpoint_epoch_{epoch+1}.pth")
    torch.save(lora_model, save_path1)
    print(f"EPOCH {epoch+1}:")
    print(f"Checkpoint saved: {save_dir}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader_train)}")
    print(f"Epoch {epoch+1}/{num_epochs}, Contrastive Loss: {contrastive_loss_total / len(data_loader_train)}")
    print(f"Epoch {epoch+1}/{num_epochs}, Entailment Loss: {entailment_loss_total / len(data_loader_train)}")
    mrr_validation = evaluate_mrr(model1=lora_model, data_loader_val=data_loader_val,distance_fn=lorentzian_distance)
    #mrr_train = evaluate_mrr(lora_model, data_loader_train, lorentzian_distance)
    #print(f"Mean Reciprocal Rank (MRR) for training set: {mrr_train:.4f}")
    print(f"Mean Reciprocal Rank (MRR) for validation set: {mrr_validation:.4f}")
    end_time = time.time()
    print(f"Epoch {epoch+1} took {(end_time - start_time) / 60:.4f} minutes.")
    print(f"\n")
    epoch_metrics.append({
        'epoch': epoch + 1,
        'training_loss': total_loss / len(data_loader_train),
        'Contrastive_loss': contrastive_loss_total / len(data_loader_train),
        'Entailment_loss': entailment_loss_total / len(data_loader_train),
        'mrr_validation': mrr_validation,
        'time_taken_minutes': (end_time - start_time) / 60
    })

EPOCH 1:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/v2/0.9hyperbolic
Epoch 1/3, Loss: 2.760899182690211
Epoch 1/3, Contrastive Loss: 1.684578627177257
Epoch 1/3, Entailment Loss: 1.1959117591510038


Mean Reciprocal Rank (MRR) for validation set: 0.6023
Epoch 1 took 216.8313 minutes.




EPOCH 2:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/v2/0.9hyperbolic
Epoch 2/3, Loss: 2.6072848108686593
Epoch 2/3, Contrastive Loss: 1.6469639575221968
Epoch 2/3, Entailment Loss: 1.0670231968312374


Mean Reciprocal Rank (MRR) for validation set: 0.6054
Epoch 2 took 216.9775 minutes.




EPOCH 3:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/v2/0.9hyperbolic
Epoch 3/3, Loss: 2.5721691094905013
Epoch 3/3, Contrastive Loss: 1.6483976306071093
Epoch 3/3, Entailment Loss: 1.0264127802815437


Mean Reciprocal Rank (MRR) for validation set: 0.6100
Epoch 3 took 217.0951 minutes.




In [41]:
import json
with open(save_dir + '/hyperbolic_epoch_metrics.json', 'w') as f:
    json.dump(epoch_metrics, f)